In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.formula.api import ols

In [2]:
data_process = pd.read_csv('houses_clean.csv',parse_dates=['Time'], index_col='Time') # load the electricity data
weather_process = pd.read_csv('weather_data.csv',parse_dates=True, index_col='dateTime')

In [3]:
weather_process.columns

Index(['Air_temperature', 'Relative_humidity', 'Wind_speed',
       'Wind_direction_D1', 'Wind_direction_SD1',
       'Average_horizontal_solar_irradiance',
       'Total_horizontal_solar_irradiation', 'NR_Wm2_Avg', 'CNR_Wm2_Avg',
       'Total_rainfall', 'Average_barometric_pressure'],
      dtype='object')

In [4]:
# Keep the the dates with less missing values for most houses
data = data_process.copy(deep=True)
data = data['2014-04':'2015-03']

# keep the same time frame for weather data and resample on hourly level by averaging
weather = weather_process.copy(deep=True)
cols = ['Wind_speed','Wind_direction_D1', 'Wind_direction_SD1', 'Average_horizontal_solar_irradiance', 'Total_horizontal_solar_irradiation', 'NR_Wm2_Avg', 'CNR_Wm2_Avg', 'Average_barometric_pressure']
weather = weather.drop(cols,axis=1)
weather = weather['2014-04':'2015-03']
weather = weather.resample('H').mean()

In [5]:
# Create seasonal variables
#data['Year'] = pd.DatetimeIndex(data.index).year
data['Month'] = pd.DatetimeIndex(data.index).month
data['Week'] = pd.DatetimeIndex(data.index).week
data['DayOfMonth'] = pd.DatetimeIndex(data.index).day
#data['DayOfWeek'] =  pd.DatetimeIndex(data.index).dayofweek
data['Day'] = pd.DatetimeIndex(data.index).weekday_name
data['Hour'] =  pd.DatetimeIndex(data.index).hour

# create weather variables
data = pd.concat([data,weather], axis=1)

In [6]:
# keep the house of choice for modelling
houses=[1,2,4,5,6,7,8,9,10,12,13,15,16,17,18,19,20]
house_nr = 1
drop_houses = ['House_'+str(i) for i in houses if i != house_nr]
data = data.drop(drop_houses, axis=1)

In [7]:
# drop nan values
data = data.dropna(axis=0,how='any')

In [8]:
data.head()

,House_1,Month,Week,DayOfMonth,Day,Hour,Air_temperature,Relative_humidity,Total_rainfall
2014-04-01 00:00:00,0.848060,4,14,1,Tuesday,0,11.7450,83.275,0.05
2014-04-01 01:00:00,0.961215,4,14,1,Tuesday,1,11.4100,85.400,0.00
2014-04-01 02:00:00,0.873575,4,14,1,Tuesday,2,10.6900,91.900,0.60
2014-04-01 03:00:00,0.893387,4,14,1,Tuesday,3,10.3250,94.225,0.00
2014-04-01 04:00:00,0.852123,4,14,1,Tuesday,4,9.9525,95.100,0.00


## Create extra features

In [9]:
# Week days Feature
data.loc[(data['Day'] == 'Monday', 'WeekDays')] = 'WeekDays'
data.loc[(data['Day'] == 'Saturday', 'WeekDays')] = 'Saturday'
data.loc[(data['Day'] == 'Sunday', 'WeekDays')] = 'Sunday'
data.loc[(data['Day'] == 'Tuesday', 'WeekDays')] = 'WeekDays'
data.loc[(data['Day'] == 'Wednesday', 'WeekDays')] = 'WeekDays'
data.loc[(data['Day'] == 'Thursday', 'WeekDays')] = 'WeekDays'
data.loc[(data['Day'] == 'Friday', 'WeekDays')] = 'WeekDays'

In [10]:
# Season feature
data.loc[ (data['Month'] == 1), 'Season'] = 'Winter'
data.loc[ (data['Month'] == 2), 'Season'] = 'Winter'
data.loc[ (data['Month'] == 12), 'Season'] = 'Winter'
data.loc[ (data['Month'] == 3),  'Season'] = 'Spring'
data.loc[ (data['Month'] == 4),  'Season'] = 'Spring'
data.loc[ (data['Month'] == 5),  'Season'] = 'Spring'
data.loc[ (data['Month'] == 6), 'Season'] = 'Summer'
data.loc[ (data['Month'] == 7), 'Season'] = 'Summer'
data.loc[ (data['Month'] == 8), 'Season'] = 'Summer'
data.loc[ (data['Month'] == 9), 'Season'] = 'Autumn' 
data.loc[ (data['Month'] == 10), 'Season'] = 'Autumn' 
data.loc[ (data['Month'] == 11), 'Season'] = 'Autumn' 

In [11]:
# Time bulk feature
time_bulk_1 = [1,2,3,4]
time_bulk_2 = [5,6,7,8]
time_bulk_3 = [9,10,11,12]
time_bulk_4 = [13,14,15,16]
time_bulk_5 = [17,18,19,20]
time_bulk_6 = [21,22,23,0]
for i in range(0,24):
    if i in time_bulk_1:
        data.loc[(data['Hour'] == i), 'Time_bulk'] = 'Midnight'
    elif i in time_bulk_2:
        data.loc[(data['Hour']== i), 'Time_bulk'] = 'Morning'
    elif i in time_bulk_3:
        data.loc[(data['Hour']== i), 'Time_bulk'] = 'Noon'
    elif i in time_bulk_4:
        data.loc[(data['Hour']== i), 'Time_bulk'] = 'Afternoon'
    elif i in time_bulk_5:
        data.loc[(data['Hour']== i), 'Time_bulk'] = 'Evening'
    else:
        data.loc[(data['Hour']== i), 'Time_bulk'] = 'Night'

In [12]:
# create trend 
data = data.assign(Trend = pd.Series(np.arange(len(data))+1).values)

In [13]:
# create trend square
x = (np.arange(data.shape[0]))**2 + 1
data = data.assign(Trend_square = pd.Series(x).values)

## Prepare data for modeling

In [14]:
data_modeling = data.copy(deep=True)

In [15]:
data_modeling.columns

Index(['House_1', 'Month', 'Week', 'DayOfMonth', 'Day', 'Hour',
       'Air_temperature', 'Relative_humidity', 'Total_rainfall', 'WeekDays',
       'Season', 'Time_bulk', 'Trend', 'Trend_square'],
      dtype='object')

In [16]:
# drop unwanted features
cols = ['Week','Day','Season','Time_bulk','Trend','Trend_square']
data_modeling = data_modeling.drop(cols, axis=1)

In [17]:
data_modeling.columns

Index(['House_1', 'Month', 'DayOfMonth', 'Hour', 'Air_temperature',
       'Relative_humidity', 'Total_rainfall', 'WeekDays'],
      dtype='object')

In [18]:
# creating dummy variables
cols =  ['DayOfMonth', 'Hour', 'WeekDays']
data_ook = pd.get_dummies(data_modeling, columns=cols, prefix=cols, drop_first=True)
data_ook = pd.get_dummies(data_ook, columns=['Month'], prefix='Month', drop_first=False)

In [19]:
# split depedent and indepedent variables
house = 'House_' + str(house_nr)
x_train = data_ook.drop(house,axis=1).values.astype(np.float)
y_train = data_ook[house].values

# Linear Regression

In [21]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [32]:
# Scale data
standard_scaler = StandardScaler()
x_train_sc = standard_scaler.fit_transform(x_train[:,0:3])

In [35]:
x_train_s = np.c_[x_train_sc, x_train[:,3:]]

In [45]:
# split data set for ts k-fold cross validation
tscv = TimeSeriesSplit(n_splits=5)

# add bias term
fit_bias = False

# Create linear regression object
lrm1 = linear_model.LinearRegression(fit_intercept= fit_bias)
lrm2 = linear_model.LinearRegression(fit_intercept= fit_bias)

# train on the whole data
pred = linear_model.LinearRegression(fit_intercept=fit_bias).fit(X = x_train_s, y= y_train).predict(x_train_s)

# cross validation
scores_mae  = cross_val_score(lrm1, x_train_s, y_train, cv=tscv, scoring='neg_mean_absolute_error')
scores_mde  = cross_val_score(lrm2, x_train_s, y_train, cv=tscv, scoring='neg_median_absolute_error')

# print scores
print('On training set, the model achieves R_2 score: {0:.3f}, MAE: {1:.3f}, MDE:{2:.3f}'.format(metrics.r2_score(y_true=y_train,y_pred=pred),metrics.mean_squared_error(y_true=y_train,y_pred=pred),metrics.median_absolute_error(y_true=y_train,y_pred=pred)))
print('On validation set, the model achieves MAE: {0:.3f}, MDE:{1:.3f}'.format(abs(scores_mae.mean()),abs(scores_mde.mean())))

On training set, the model achieves R_2 score: 0.151, MAE: 0.110, MDE:0.160
On validation set, the model achieves MAE: 0.338, MDE:0.237


In [47]:
data_modeling.columns

Index(['House_1', 'Month', 'DayOfMonth', 'Hour', 'Air_temperature',
       'Relative_humidity', 'Total_rainfall', 'WeekDays'],
      dtype='object')

In [65]:
model_3 = ols(formula='House_1 ~ Relative_humidity + C(Month) + C(DayOfMonth) + C(WeekDays) + C(Hour) -1', data=data, missing='drop').fit()
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:                House_1   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     22.51
Date:                Thu, 22 Jun 2017   Prob (F-statistic):          5.64e-247
Time:                        01:51:33   Log-Likelihood:                -2696.4
No. Observations:                8524   AIC:                             5529.
Df Residuals:                    8456   BIC:                             6008.
Df Model:                          67                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
C(Month)[1]               

In [66]:
print(sm.stats.anova_lm(model_3, typ=2))

                       sum_sq      df          F         PR(>F)
C(Month)            94.603820    12.0  70.952075  5.092698e-166
C(DayOfMonth)       25.295490    30.0   7.588563   9.363121e-32
C(WeekDays)          2.968518     2.0  13.358182   1.613469e-06
C(Hour)             47.135465    23.0  18.444107   1.893948e-73
Relative_humidity    1.124291     1.0  10.118507   1.473182e-03
Residual           939.566008  8456.0        NaN            NaN
